In [ ]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00


In [ ]:
pip install torch_geometric_temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import precision_recall_curve, confusion_matrix

# --- Device Configuration ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- Utility Function for Edge Creation (MOVE THIS BLOCK HERE) ---
def create_edge_index_from_series(card_series):
    edge_index = []
    col_to_group_by = card_series.name if card_series.name is not None else 'value' # Default name

    temp_df = card_series.reset_index()
    # If `col_to_group_by` is not found, it implies the default 'value' for reset_index() series
    if col_to_group_by not in temp_df.columns:
        col_to_group_by = temp_df.columns[1] # Assumes 'index' is 0, values are 1 for reset_index()

    card_to_indices = temp_df.groupby(col_to_group_by)['index'].apply(list)

    for card_val, indices in card_to_indices.items():
        if pd.notna(card_val):
            # Only create edges if there are at least two transactions with the same value
            if len(indices) > 1:
                for i in range(len(indices)):
                    for j in range(i + 1, len(indices)):
                        # Add undirected edges
                        edge_index.append([indices[i], indices[j]])
                        edge_index.append([indices[j], indices[i]])
    if not edge_index:
        print(f"Warning: No edges created for grouping by '{col_to_group_by}'. Returning empty edge_index.")
        return torch.empty((2, 0), dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return edge_index


# Step 1: Load the Data - MODIFIED FOR creditcard.csv
# Assuming 'creditcard.csv' is in the /content/ directory or accessible

data_df = None # Initialize data_df to None

try:
    data_df = pd.read_csv(r'/content/creditcard.csv')
    print("Successfully loaded 'creditcard.csv'.")
except FileNotFoundError:
    print("Error: 'creditcard.csv' not found. Please ensure the file is in the '/content/' directory.")
    print("You might need to upload it to Colab, or mount Google Drive and adjust the path.")
    raise # Re-raise the exception to stop execution gracefully.
except Exception as e:
    print(f"An unexpected error occurred while loading the CSV: {e}")
    raise # Re-raise any other exceptions

# Now, we are sure data_df is loaded if we reach here
# Rename 'Class' to 'isFraud' for consistency with the rest of your code
if 'Class' in data_df.columns:
    data_df.rename(columns={'Class': 'isFraud'}, inplace=True)
    print("Renamed 'Class' column to 'isFraud'.")
else:
    print("Warning: 'Class' column (expected target) not found. Please ensure the target column is named 'isFraud' or 'Class'.")


# We need a unique identifier for the test set. 'Time' might not be unique if transactions
# occur at the exact same second. Let's create a 'TransactionID' column based on the index
# if it doesn't exist, to ensure a stable merge for the submission.
if 'TransactionID' not in data_df.columns:
    data_df['TransactionID'] = data_df.index
    print("Created 'TransactionID' column from DataFrame index.")

# To simulate train/test split like the original problem, we'll sort by 'Time'
# and then split. The original problem used 'TransactionDT' for time.
# We'll use the 'Time' column from 'creditcard.csv' for sorting.
if 'Time' in data_df.columns:
    data_df = data_df.sort_values(by='Time').reset_index(drop=True)
    print("Sorted data by 'Time' column.")
else:
    print("Warning: 'Time' column not found. Data will not be sorted by time before splitting.")


# Define the split point for train and test (e.g., 80% for train, 20% for test)
train_split_ratio = 0.8
split_point = int(len(data_df) * train_split_ratio)

train = data_df.iloc[:split_point].copy()
test = data_df.iloc[split_point:].copy()

# Store original test TransactionIDs for submission alignment
test_transaction_ids = test['TransactionID'].copy()

gc.collect()

print(f"Train shape: {train.shape}, Test shape: {test.shape}")

# Step 2: Enhanced Preprocessing
missing_percent = train.isnull().mean()
high_missing_cols = missing_percent[missing_percent > 0.8].index.tolist()
if high_missing_cols:
    train.drop(columns=high_missing_cols, inplace=True)
    test.drop(columns=high_missing_cols, inplace=True)
    print(f"Removed columns with >80% missing values: {high_missing_cols}")
else:
    print("No columns with >80% missing values found.")

potential_categorical_cols = []
categorical_cols = [col for col in potential_categorical_cols if col in train.columns]
missing_cols = [col for col in potential_categorical_cols if col not in train.columns]
if missing_cols:
    print(f"Note: The following expected categorical columns are missing in the dataset: {missing_cols}")
if categorical_cols:
    print(f"Identified categorical columns: {categorical_cols}")
else:
    print("No predefined categorical columns found in the dataset (common for creditcard.csv).")

numerical_cols_initial = train.select_dtypes(include=['float64', 'int64']).columns.tolist()
if 'isFraud' in numerical_cols_initial:
    numerical_cols_initial.remove('isFraud')
if 'TransactionID' in numerical_cols_initial:
    numerical_cols_initial.remove('TransactionID')
if 'Time' in numerical_cols_initial:
    numerical_cols_initial.remove('Time')

print("Filling missing numerical values with median (if any)...")
for col in numerical_cols_initial:
    if train[col].isnull().any() or test[col].isnull().any():
        median_val = train[col].median()
        train[col] = train[col].fillna(median_val)
        test[col] = test[col].fillna(median_val)
        print(f"Filled NaNs in {col} with median {median_val}.")

outlier_cols_to_check = ['Amount']
if 'Amount' in train.columns:
    print("Applying IQR-based outlier removal for 'Amount' in training data...")
    col = 'Amount'
    q1 = train[col].quantile(0.05)
    q3 = train[col].quantile(0.95)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    original_train_rows = train.shape[0]
    train = train[(train[col] >= lower_bound) & (train[col] <= upper_bound)].copy()
    print(f"Removed {original_train_rows - train.shape[0]} outlier rows based on 'Amount'. New train shape: {train.shape}")
else:
    print("No 'Amount' column found for IQR outlier removal.")


# Step 3: Enhanced Feature Engineering
if 'Time' in train.columns:
    train.rename(columns={'Time': 'TransactionDT'}, inplace=True)
    test.rename(columns={'Time': 'TransactionDT'}, inplace=True)
    print("Renamed 'Time' to 'TransactionDT' for time-based features.")

    train['hour'] = ((train['TransactionDT'] // 3600) % 24)
    test['hour'] = ((test['TransactionDT'] // 3600) % 24)
    print("Created 'hour' feature from 'TransactionDT'.")
else:
    print("No 'Time'/'TransactionDT' column found for time-based feature engineering.")

if 'Amount' in train.columns:
    train['LogTransactionAmt'] = np.log1p(train['Amount'])
    test['LogTransactionAmt'] = np.log1p(test['Amount'])
    print("Created 'LogTransactionAmt' from 'Amount'.")
else:
    print("No 'Amount' column found for Log transformation.")

print("Skipping email domain grouping (not applicable for creditcard.csv).")
print("Skipping card frequency features (not applicable for creditcard.csv with original columns).")
for col in ['card1', 'card2', 'card3', 'card5']:
    if col in train.columns:
        freq_map = train[col].value_counts().to_dict()
        train[f'{col}_freq'] = train[col].map(freq_map)
        test[f'{col}_freq'] = test[col].map(freq_map)
        train[f'{col}_freq'] = train[f'{col}_freq'].fillna(0)
        test[f'{col}_freq'] = test[f'{col}_freq'].fillna(0)

print("Skipping device info features (not applicable for creditcard.csv).")


# Step 4: Prepare Data for GNN and LSTM
numerical_cols = ['LogTransactionAmt', 'hour'] if 'LogTransactionAmt' in train.columns and 'hour' in train.columns else []

v_cols = [f'V{i}' for i in range(1, 29) if f'V{i}' in train.columns]
numerical_cols.extend(v_cols)

if 'Amount' in train.columns and 'LogTransactionAmt' not in numerical_cols:
    numerical_cols.append('Amount')

m_cols = [f'M{i}' for i in range(1, 10) if f'M{i}' in train.columns]
for col in m_cols:
    if col in train.columns:
        train[col] = train[col].map({'T': 1, 'F': 0}).astype(float)
        test[col] = test[col].map({'T': 1, 'F': 0}).astype(float)
        train[col].fillna(0, inplace=True)
        test[col].fillna(0, inplace=True)
    numerical_cols.append(col)

feature_selection_categorical_cols = []

label_encoders = {}
for col in feature_selection_categorical_cols:
    le = LabelEncoder()
    train[col] = train[col].fillna('missing').astype(str)
    test[col] = test[col].fillna('missing').astype(str)
    combined = pd.concat([train[col], test[col]], axis=0)
    le.fit(combined)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le
    print(f"Label encoded column: {col}")


features = numerical_cols + feature_selection_categorical_cols
features = [f for f in features if f in train.columns]
if not features:
    raise ValueError("No features selected for training. Please check feature definitions.")
print(f"Selected features for model: {features}")


print("Checking and filling any remaining NaNs in selected features...")
for col in features:
    if train[col].isnull().any():
        if col in numerical_cols:
            median_val = train[col].median()
            train[col].fillna(median_val, inplace=True)
            test[col].fillna(median_val, inplace=True)
            print(f"Filled remaining NaNs in {col} (numerical) with median {median_val}.")
        elif col in feature_selection_categorical_cols:
            train[col].fillna(0, inplace=True)
            test[col].fillna(0, inplace=True)
            print(f"Filled remaining NaNs in {col} (categorical) with 0.")
        else:
            if pd.api.types.is_numeric_dtype(train[col]):
                median_val = train[col].median()
                train[col].fillna(median_val, inplace=True)
                test[col].fillna(median_val, inplace=True)
                print(f"Filled remaining NaNs in {col} (unknown numeric) with median {median_val}.")
            else:
                train[col].fillna('unknown_nan_val', inplace=True)
                test[col].fillna('unknown_nan_val', inplace=True)
                print(f"Filled remaining NaNs in {col} (unknown type) with 'unknown_nan_val'.")


print(f"NaNs in train[features] before scaling: {train[features].isnull().sum().sum()}")
print(f"NaNs in test[features] before scaling: {test[features].isnull().sum().sum()}")

X = train[features]
y = train['isFraud']
X_test_full = test[features]

print("Performing L1-Regularized Logistic Regression for Feature Importance...")
temp_scaler = StandardScaler()
X_temp_scaled = temp_scaler.fit_transform(X)
logistic_reg = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=42, class_weight='balanced', max_iter=1000)
logistic_reg.fit(X_temp_scaled, y)

feature_importance = pd.DataFrame({'Feature': features, 'Coefficient': logistic_reg.coef_[0]})
feature_importance['Abs_Coefficient'] = np.abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)
print("Top 20 Feature Importances (L1 Logistic Regression):")
print(feature_importance.head(20))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_full)

minmax_scaler = MinMaxScaler(feature_range=(0, 0.7))
numerical_indices = [features.index(col) for col in numerical_cols if col in features]
X_scaled[:, numerical_indices] = minmax_scaler.fit_transform(X_scaled[:, numerical_indices])
X_test_scaled[:, numerical_indices] = minmax_scaler.transform(X_test_scaled[:, numerical_indices])

original_num_train_nodes = X_scaled.shape[0]
original_y = y.copy()

print("\n--- Training GAN for Data Augmentation ---")

X_fraud_real = X_scaled[y == 1]
X_non_fraud_real = X_scaled[y == 0]

class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, output_dim),
            nn.Sigmoid()
        )
    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2, True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, input):
        return self.main(input)

latent_dim = 100
gan_output_dim = X_scaled.shape[1]
lr_gan = 0.0002
b1 = 0.5
b2 = 0.999
n_epochs_gan = 3000
batch_size_gan = 64

generator = Generator(latent_dim, gan_output_dim).to(device)
discriminator = Discriminator(gan_output_dim).to(device)

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_gan, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_gan, betas=(b1, b2))

adversarial_loss = nn.BCELoss()

real_fraud_data_tensor = torch.tensor(X_fraud_real, dtype=torch.float).to(device)

for epoch in range(n_epochs_gan):
    optimizer_D.zero_grad()
    real_idx = torch.randint(0, real_fraud_data_tensor.size(0), (batch_size_gan,))
    real_samples = real_fraud_data_tensor[real_idx]
    real_labels = torch.ones(batch_size_gan, 1).to(device)
    d_output_real = discriminator(real_samples)
    d_loss_real = adversarial_loss(d_output_real, real_labels)

    z = torch.randn(batch_size_gan, latent_dim).to(device)
    fake_samples = generator(z).detach()
    fake_labels = torch.zeros(batch_size_gan, 1).to(device)
    d_output_fake = discriminator(fake_samples)
    d_loss_fake = adversarial_loss(d_output_fake, fake_labels)

    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    optimizer_D.step()

    optimizer_G.zero_grad()
    z = torch.randn(batch_size_gan, latent_dim).to(device)
    gen_samples = generator(z)
    g_output = discriminator(gen_samples)
    g_loss = adversarial_loss(g_output, real_labels)
    g_loss.backward()
    optimizer_G.step()

    if (epoch + 1) % 500 == 0:
        print(f"GAN Epoch {epoch+1}/{n_epochs_gan}, D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

print("GAN training finished.")

num_synthetic_samples_needed = len(X_non_fraud_real) - len(X_fraud_real)
if num_synthetic_samples_needed < 0:
    num_synthetic_samples_needed = 0

print(f"Generating {num_synthetic_samples_needed} synthetic fraud samples...")

generator.eval()
with torch.no_grad():
    synthetic_z = torch.randn(num_synthetic_samples_needed, latent_dim).to(device)
    X_synthetic_fraud = generator(synthetic_z).cpu().numpy()

X_augmented_fraud = np.vstack([X_fraud_real, X_synthetic_fraud])
y_augmented_fraud = np.ones(X_augmented_fraud.shape[0])

X_resampled_gan = np.vstack([X_non_fraud_real, X_augmented_fraud])
y_resampled_gan = np.hstack([np.zeros(X_non_fraud_real.shape[0]), y_augmented_fraud])

shuffled_indices = np.random.permutation(len(X_resampled_gan))
X_resampled_gan = X_resampled_gan[shuffled_indices]
y_resampled_gan = y_resampled_gan[shuffled_indices]

print(f"Augmented data shape (X): {X_resampled_gan.shape}")
print(f"Augmented data shape (y): {y_resampled_gan.shape}")
print(f"Fraud count in augmented data: {np.sum(y_resampled_gan == 1)}")
print(f"Non-Fraud count in augmented data: {np.sum(y_resampled_gan == 0)}")


# 4.6 Create graph structure (edges based on shared card1) using augmented data
node_grouping_feature = None
# Prioritize 'TransactionID' as a unique identifier if we created it.
# Otherwise, default to 'V1' as a numerical feature.
if 'TransactionID' in features:
    node_grouping_feature = 'TransactionID'
elif 'V1' in features:
    node_grouping_feature = 'V1'
elif 'Time' in features: # Use 'TransactionDT' if 'Time' was renamed
    node_grouping_feature = 'TransactionDT'
else:
    print("Warning: No suitable feature for graph edge creation found. Graph might be empty or problematic.")

if node_grouping_feature:
    print(f"Using '{node_grouping_feature}' for graph edge creation.")
    grouping_idx = features.index(node_grouping_feature)
    temp_grouping_train = pd.Series(X_resampled_gan[:, grouping_idx], name=node_grouping_feature)
    temp_grouping_test = pd.Series(X_test_scaled[:, grouping_idx], name=node_grouping_feature)

    train_edge_index = create_edge_index_from_series(temp_grouping_train)
    test_edge_index = create_edge_index_from_series(temp_grouping_test)
else:
    print("WARNING: No suitable grouping feature for GNN edge creation. Creating an empty edge index. GNN may not function as intended.")
    train_edge_index = torch.empty((2, 0), dtype=torch.long)
    test_edge_index = torch.empty((2, 0), dtype=torch.long)


# 4.7 Create PyTorch Geometric Data object for training using GAN augmented data
x_train_gan = torch.tensor(X_resampled_gan, dtype=torch.float)
y_train_gan = torch.tensor(y_resampled_gan, dtype=torch.long)
data = Data(x=x_train_gan, edge_index=train_edge_index, y=y_train_gan)
print(f"PyG Data object created for training. Num nodes: {data.num_nodes}, Num edges: {data.num_edges}")


# 4.8 Create train/val/test masks - these will now be over the GAN-augmented data
n_samples_gan = len(y_resampled_gan)
train_idx, temp_idx = train_test_split(range(n_samples_gan), test_size=0.3, random_state=42, stratify=y_resampled_gan)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42, stratify=y_resampled_gan[temp_idx])

train_mask = torch.zeros(n_samples_gan, dtype=torch.bool)
val_mask = torch.zeros(n_samples_gan, dtype=torch.bool)
test_mask = torch.zeros(n_samples_gan, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

print(f"Train mask size: {train_mask.sum()}, Val mask size: {val_mask.sum()}, Test mask size: {test_mask.sum()}")


# 4.9 Create PyTorch Geometric Data object for test set (original, not augmented)
test_data = Data(x=torch.tensor(X_test_scaled, dtype=torch.float), edge_index=test_edge_index)
print(f"PyG Data object created for test set. Num nodes: {test_data.num_nodes}, Num edges: {test_data.num_edges}")


# Step 5: Define Temporal GNN Model with LSTM
class TemporalGCNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_gnn_dim, hidden_lstm_dim, output_dim, sequence_length, dropout_rate=0.7):
        super(TemporalGCNLSTM, self).__init__()
        self.sequence_length = sequence_length
        self.hidden_gnn_dim = hidden_gnn_dim
        self.dropout_rate = dropout_rate

        self.conv1 = GCNConv(input_dim, hidden_gnn_dim)
        self.bn1 = nn.BatchNorm1d(hidden_gnn_dim)
        self.conv2 = GCNConv(hidden_gnn_dim, hidden_gnn_dim)
        self.bn2 = nn.BatchNorm1d(hidden_gnn_dim)

        self.lstm = nn.LSTM(hidden_gnn_dim, hidden_lstm_dim, batch_first=True)

        self.fc = nn.Linear(hidden_lstm_dim, output_dim)

    def forward(self, x, edge_index, batch_size=1):
        x_gnn = self.conv1(x, edge_index)
        x_gnn = self.bn1(x_gnn)
        x_gnn = F.relu(x_gnn)
        x_gnn = F.dropout(x_gnn, p=self.dropout_rate, training=self.training)
        x_gnn = self.conv2(x_gnn, edge_index)
        x_gnn = self.bn2(x_gnn)
        x_gnn = F.relu(x_gnn)
        x_gnn = F.dropout(x_gnn, p=self.dropout_rate, training=self.training)

        x_lstm_input = x_gnn.unsqueeze(0)

        lstm_out, (h_n, c_n) = self.lstm(x_lstm_input)

        out = self.fc(lstm_out.squeeze(0))

        return F.log_softmax(out, dim=1)


# Step 6: Train Temporal GNN Model
input_dim = X_resampled_gan.shape[1]
hidden_gnn_dim = 96
hidden_lstm_dim = 96
output_dim = 2
sequence_length = len(X_resampled_gan)
dropout_rate = 0.7

model = TemporalGCNLSTM(input_dim=input_dim,
                        hidden_gnn_dim=hidden_gnn_dim,
                        hidden_lstm_dim=hidden_lstm_dim,
                        output_dim=output_dim,
                        sequence_length=sequence_length,
                        dropout_rate=dropout_rate).to(device)

data = data.to(device)
test_data = test_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

class_weights = torch.tensor([1.0, (y_train_gan == 0).sum() / (y_train_gan == 1).sum()], dtype=torch.float).to(device)
criterion = nn.NLLLoss(weight=class_weights)
print(f"Calculated class weights for NLLLoss: {class_weights}")


def train_model():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate_model(mask_or_none, data_obj, optimal_threshold=None, is_original_test_set=False):
    model.eval()
    with torch.no_grad():
        out = model(data_obj.x, data_obj.edge_index)

        if is_original_test_set:
            probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()
            return probs
        else:
            probs = torch.softmax(out[mask_or_none], dim=1)[:, 1].cpu().numpy()
            true = data_obj.y[mask_or_none].cpu().numpy()

            if optimal_threshold is None:
                precision_curve, recall_curve, thresholds = precision_recall_curve(true, probs)
                f1_scores = 2 * (precision_curve * recall_curve) / (precision_curve + recall_curve + 1e-9)

                if len(f1_scores) > 0 and len(thresholds) > 0:
                    optimal_idx = np.argmax(f1_scores)
                    optimal_threshold_found = thresholds[optimal_idx]
                else:
                    optimal_threshold_found = 0.5
                return optimal_threshold_found, probs, true
            else:
                pred = (probs >= optimal_threshold).astype(int)
                precision = precision_score(true, pred, zero_division=0)
                recall = recall_score(true, pred, zero_division=0)
                f1 = f1_score(true, pred, zero_division=0)
                auc = roc_auc_score(true, probs)
                accuracy = accuracy_score(true, pred)

                tn, fp, fn, tp = confusion_matrix(true, pred).ravel()

                sensitivity = tp / (tp + fn + 1e-9)
                specificity = tn / (tn + fp + 1e-9)

                g_mean_sensitivity = np.sqrt(sensitivity * specificity)

                return precision, recall, f1, auc, accuracy, sensitivity, specificity, g_mean_sensitivity

patience = 30
best_val_f1 = -1
epochs_no_improve = 0
max_epochs = 400

print("\nTraining TGNN+LSTM with GAN-augmented data and early stopping...")
for epoch in range(1, max_epochs + 1):
    train_loss = train_model()

    current_optimal_threshold, val_probs_epoch, val_true_epoch = evaluate_model(
        data.val_mask, data, optimal_threshold=None, is_original_test_set=False
    )

    precision_curve, recall_curve, thresholds = precision_recall_curve(val_true_epoch, val_probs_epoch)
    f1_scores = 2 * (precision_curve * recall_curve) / (precision_curve + recall_curve + 1e-9)
    current_val_f1 = np.max(f1_scores) if len(f1_scores) > 0 else 0.0

    if current_val_f1 > best_val_f1:
        best_val_f1 = current_val_f1
        best_optimal_threshold = current_optimal_threshold
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1

    if epoch % 20 == 0 or epochs_no_improve == 0:
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Val F1: {current_val_f1:.4f}, Best Val F1: {best_val_f1:.4f}, Epochs no improve: {epochs_no_improve}')

    if epochs_no_improve == patience:
        print(f"Early stopping at epoch {epoch} as validation F1 did not improve for {patience} epochs.")
        break

print(f"Optimal threshold found on validation set: {best_optimal_threshold:.4f}")

model.load_state_dict(torch.load('best_model.pth'))

test_precision, test_recall, test_f1, test_auc, test_accuracy, test_sensitivity, test_specificity, test_g_mean_sensitivity = evaluate_model(
    data.test_mask, data, best_optimal_threshold, is_original_test_set=False
)
print("\nInternal (GAN-augmented) Test Set Performance:")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall (Sensitivity): {test_recall:.4f}")
print(f"Test Specificity: {test_specificity:.4f}")
print(f"Test F1: {test_f1:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test G-Mean Sensitivity: {test_g_mean_sensitivity:.4f}")


train_precision, train_recall, train_f1, train_auc, train_accuracy, train_sensitivity, train_specificity, train_g_mean_sensitivity = evaluate_model(
    data.train_mask, data, best_optimal_threshold, is_original_test_set=False
)
print("\nInternal (GAN-augmented) Train Set Performance:")
print(f"Train Precision: {train_precision:.4f}")
print(f"Train Recall (Sensitivity): {train_recall:.4f}")
print(f"Train Specificity: {train_specificity:.4f}")
print(f"Train F1: {train_f1:.4f}")
print(f"Train AUC: {train_auc:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Train G-Mean Sensitivity: {train_g_mean_sensitivity:.4f}")


print("\nGenerating predictions for the actual unseen test data (submission file)...")
submission_probs = evaluate_model(
    None,
    test_data,
    best_optimal_threshold,
    is_original_test_set=True
)

try:
    final_submission_df = sample_submission[['TransactionID']].merge(
        pd.DataFrame({'TransactionID': test_transaction_ids, 'isFraud': submission_probs}),
        on='TransactionID', how='left'
    )
    print("Using provided sample_submission for merge.")
except NameError:
    print("`sample_submission` not defined. Creating submission DataFrame directly.")
    final_submission_df = pd.DataFrame({
        'TransactionID': test_transaction_ids,
        'isFraud': submission_probs
    })
    final_submission_df.rename(columns={'TransactionID': 'id'}, inplace=True)
except KeyError:
    print("`sample_submission` missing 'TransactionID' column. Creating submission DataFrame directly.")
    final_submission_df = pd.DataFrame({
        'TransactionID': test_transaction_ids,
        'isFraud': submission_probs
    })
    final_submission_df.rename(columns={'TransactionID': 'id'}, inplace=True)


final_submission_df['isFraud'] = final_submission_df['isFraud'].fillna(0)

final_submission_df.to_csv('submission.csv', index=False)
print("Submission file generated: submission.csv")
print("Note: Performance metrics (Precision, Recall, F1, AUC, Accuracy, G-Mean Sensitivity) are not reported for the final 'test.csv' dataset as its true labels are unknown.")

Using device: cpu
Successfully loaded 'creditcard.csv'.
Renamed 'Class' column to 'isFraud'.
Created 'TransactionID' column from DataFrame index.
Sorted data by 'Time' column.
Train shape: (12748, 32), Test shape: (3188, 32)
No columns with >80% missing values found.
No predefined categorical columns found in the dataset (common for creditcard.csv).
Filling missing numerical values with median (if any)...
Filled NaNs in V23 with median -0.044410388486525496.
Filled NaNs in V24 with median 0.0689371724290234.
Filled NaNs in V25 with median 0.1526519343664955.
Filled NaNs in V26 with median -0.02015846823294905.
Filled NaNs in V27 with median -0.0008983923168723.
Filled NaNs in V28 with median 0.015861526966713953.
Filled NaNs in Amount with median 15.364999999999998.
Applying IQR-based outlier removal for 'Amount' in training data...
Removed 159 outlier rows based on 'Amount'. New train shape: (12589, 32)
Renamed 'Time' to 'TransactionDT' for time-based features.
Created 'hour' feature 